# Advanced Recommendation System

## Intro

### The Data

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip) or just use the u.data file

In [1]:
import numpy as np
import pandas as pd

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


We can use the Movie_Id_Titles to grab the movie names and merge it with the dataframe.

In [4]:
movie_titles = pd.read_csv('Movie_Id_Titles')
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
df = pd.merge(df, movie_titles, on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Number of Users: '+str(n_users))
print('Number of Movies: '+str(n_items))

Number of Users: 944
Number of Movies: 1682


### Train Test Split

We'll use the same structure that we have used before to evaluate the data. However, we won't do the classic X_train, y_train, ...

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_data, test_data = train_test_split(df, test_size=0.25)

## Memory Based Collaborative Filtering

Memory Based CF can be divided into **user-item** and **item-item** filtering.

- *Item-Item*: "Users who liked this item also liked...
- *User-Item*: "Users who are similar to you also liked...

In both cases, you created a user-item matrix built from the entire dataset. 

The similarity values between items in *item-item* are measured by observing all the users who have rated both items.

The similarity values between users in *user-item* are measured by observing all the items rated by both users.

A distance metric commonly used in recommender system is *cosine similarity*, where ratings are considered vectors in `n`-dimensional space and the similarity is calculated based on the angle between these vectors.
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

The first step will be to create the user-item matrices.

In [9]:
train_data_matrix = np.zeros((n_users, n_items))
test_data_matrix = np.zeros((n_users, n_items))

In [10]:
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]

In [11]:
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

We can use pairwise_distances from sklearn to calculate the cosine similarity.

In [12]:
from sklearn.metrics.pairwise import pairwise_distances

In [13]:
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

These matrices enables us to make predictions by applying the following formula:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

You can look at the similarity between users *k* and *a* as weights that are multiplied by the ratings of a similar user *a*, corrected for the average rating of that user.

Some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values.

When making a prediction for **item-based CF** there is no need to correct for user's average rating.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [14]:
def predict(ratings, similarity, flag='user'):
    if flag == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has the same format.
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif flag == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

And now we apply the predict formula in each case

In [15]:
item_prediction = predict(train_data_matrix, item_similarity, flag='item')
user_prediction = predict(train_data_matrix, user_similarity, flag='user')

### Evaluation

One of the most popular metric used to evaluate accuracy of predicted ratings is *Root Mean Squared Error* (RMSE).

<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

We can use the function from `sklearn` (see more [here](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf)).

Since we want to consider predict ratings in the test dataset, we filter out all other elements in the prediction matrix using `prediction[ground_truth.nonzero()]`:

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [17]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [19]:
print('User-based CF RMSE: '+str(rmse(user_prediction, test_data_matrix)))
print('User-based CF RMSE: '+str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1197490584944862
User-based CF RMSE: 3.4456942938544026


Memory-based algorithms are easy to implement and produce reasonable prediction quality. However, they don't scale to real-world scenarios and do not address the well-known *cold-start* problem: what happens when a new user/item enters the system?

See this [post](http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/) about Memory-Based Collaborative Filtering. 

## Model-Based Collaborative Filtering

Model-based CF is based on **Matrix Factorization**. Its main goal is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings), to then predict the unknown ratings through the dot product of the latent features of users and items.

It tries to answer the question: **What does this kind of user values in this kind of item?**

When there is a sparse matrix (lots of dimensions), by doing matrix factorization you can restructure the user-item matrix into low-rank structure, and you can represent the matrix by the multiplication of two low-rank matrices, where the rows contain the latent vector. You fit the matrix to approximate your original matrix as closely as possible, by multiplying the low rank matrices together, filling the entries missing in the original matrix.

In [20]:
sparsity = round(1.0 - len(df) / float(n_users * n_items), 3)

In [22]:
print('The sparsity level of MovieLens100K is : '+str(sparsity*100) + '%')

The sparsity level of MovieLens100K is : 93.7%


The important aspect is that the model only uses data to learn the latent features. If there is little data available, the model will predict poorly.

### SVD

A well known matrix factorization method is **Singular Value Decomposition** (SVD): Read [this](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) article and also [this](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf) one.

The general equation can be expressed as follows:

<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />

Given `m x n` matrix `X`:

- *`U`* is an `m x r` orthogonal matrix
- *`S`* is an `r x r` diagonal matrix with non-negative real values.
- *V^T* is an `r x n` orthogonal matrix

Elements on the diagonal in `S` are known as *singular values* of `X`.

Matrix `X` can be factorized to `U`, `S`and `V`. The `U` matrix represents the feature vectors corresponding to the users in the hidden feature, and the `V` matrix represents the feature vectors corresponding to the items in the hidden feature space.

Now we can make predictions by taking dot product of `U`, `S` and `V^T`.

In [24]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#Get SVD components from train matrix.
# Choose k to be 20

u, s, vt = svds(train_data_matrix, k=20)
s_diag_matrix = np.diag(s)

X_prediction = np.dot(np.dot(u, s_diag_matrix), vt)

print('User-based MSE: '+str(rmse(X_prediction, test_data_matrix)))

User-based MSE: 2.720030972978146


SVD can be very slow and computationally expensive. Alternating least square and stochastic gradient descent methods will be covered in detail in the future.

## Looking for more?

If you want to tackle your own recommendation system analysis, check out these data sets. Note: The files are quite large in most cases, not all the links may stay up to host the data, but the majority of them still work. Or just Google for your own data set!

**Movies Recommendation:**

MovieLens - Movie Recommendation Data Sets http://www.grouplens.org/node/73

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Jester - Movie Ratings Data Sets (Collaborative Filtering Dataset) http://www.ieor.berkeley.edu/~goldberg/jester-data/

Cornell University - Movie-review data for use in sentiment-analysis experiments http://www.cs.cornell.edu/people/pabo/movie-review-data/

**Music Recommendation:**

Last.fm - Music Recommendation Data Sets http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Audioscrobbler - Music Recommendation Data Sets http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

Amazon - Audio CD recommendations http://131.193.40.52/data/

**Books Recommendation:**

Institut für Informatik, Universität Freiburg - Book Ratings Data Sets http://www.informatik.uni-freiburg.de/~cziegler/BX/
Food Recommendation:

Chicago Entree - Food Ratings Data Sets http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data
Merchandise Recommendation:

**Healthcare Recommendation:**

Nursing Home - Provider Ratings Data Set http://data.medicare.gov/dataset/Nursing-Home-Compare-Provider-Ratings/mufm-vy8d

Hospital Ratings - Survey of Patients Hospital Experiences http://data.medicare.gov/dataset/Survey-of-Patients-Hospital-Experiences-HCAHPS-/rj76-22dk

**Dating Recommendation:**

www.libimseti.cz - Dating website recommendation (collaborative filtering) http://www.occamslab.com/petricek/data/
Scholarly Paper Recommendation:

National University of Singapore - Scholarly Paper Recommendation http://www.comp.nus.edu.sg/~sugiyama/SchPaperRecData.html
